# Data Engineering Notebook

## Setup

In [ ]:
%load_ext autoreload
%autoreload 2

#### Terminology 

> Data engineering can consist of ***collection, cleaning, transformation, processing, and automating and monitoring tasks***
* Collection - examples include getting data from a rest service as a
* Cleaning - categorizing 
* Transformation - cateogorizing, standardization, 
* Processing - algorithm, pivot, groupby, merge
* Automating - schedule task, Apache Airflow

> Planning Jargon
* ADU - Accessory Dwelling Unit
* Existing Development Right - refers to residential, commercial, or tourist development currently built in the Lake Tahoe Basin

#### Packages, Maps, and Reference Data

In [ ]:
import pandas as pd
import numpy as np
import os
from utils import *
import getpass
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.mapping import show_styles, display_colormaps
from arcgis.gis import GIS

***Pandas Options***

In [ ]:
# set data frame display options
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 1000)
pd.options.display.float_format = '{:,.2f}'.format

***Map Setup***

In [ ]:
# Set up the GIS object
## portal URL = "https://maps.trpa.org/portal/home/"
## AGOL URL   = "https://www.arcgis.com"
gis = GIS(
    url="https://maps.trpa.org/portal/home/",
    ## enter username above ##
    username= input("Enter username:"),
    ## enter password above ##
    password=getpass.getpass("Enter password:")
)

In [ ]:
# make a map object
map = gis.map("Lake Tahoe", zoomlevel=10)

In [ ]:
map

***Get Reference Data***
* https://www.laketahoeinfo.org/WebServices/List
* https://maps.trpa.org/server/rest/services/

In [ ]:
## LT Info Data
# Verified Development Rights
dfDevRight  = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetParcelDevelopmentRightsForAccela/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
# Deed Restrictions as a DataFrame
dfDeed      = pd.read_json("https://laketahoeinfo.org/WebServices/GetDeedRestrictedParcels/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
# IPES LTinfo as a DataFrame
dfIPES      = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetParcelIPESScores/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
# Development Rights Transacted and Banked as a DataFrame
dfDevRights = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetTransactedAndBankedDevelopmentRights/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
# All Parcels as a DataFrame
dfLTParcel  = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetAllParcels/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")

In [ ]:
## TRPA Data 
# Parcel Master as a Spatially Enabled Dataframe from a Feature Service
sdfParcel     = get_fs_data_spatial("https://maps.trpa.org/server/rest/services/Parcels/FeatureServer/0")

In [ ]:
## TRPA Data 
# Parcel Master as a Spatially Enabled Dataframe from a Feature Service
sdfParcel     = get_fs_data_spatial("https://maps.trpa.org/server/rest/services/Parcels/FeatureServer/0")
# TRPA Boundary as a Spatially Enabled Dataframe from a Feature Service
sdfBoundary   = get_fs_data_spatial("https://maps.trpa.org/server/rest/services/Boundaries/FeatureServer/4")
# Plan Area Boundary as a Spatially Enabled Dataframe from a Feature Service
sdfPlanArea   = get_fs_data_spatial("https://maps.trpa.org/server/rest/services/Boundaries/FeatureServer/0")
# District Boundary as a Spatially Enabled Dataframe from a Feature Service
sdfDistrict   = get_fs_data_spatial("https://maps.trpa.org/server/rest/services/Zoning/FeatureServer/0")
# Town Center Boundary as a Spatially Enabled Dataframe from a Feature Service
sdfTownCenter = get_fs_data_spatial("https://maps.trpa.org/server/rest/services/Boundaries/FeatureServer/1")

***Testing***

In [ ]:
sdfParcel.spatial

In [ ]:
sdfParcel.spatial.full_extent

In [ ]:
sdfParcel.spatial.project(4326)

In [ ]:
sdfParcel.spatial.sr

In [ ]:
sdfParcel.spatial.full_extent

In [ ]:
sdfParcel.spatial.plot(map)

In [ ]:
sdfParcel.spatial.join(sdfBoundary, how='left', op='intersects')

In [ ]:
sdfParcel.spatial.overlay(sdfBoundary, how='intersection')

## Permit Data Engineering

#### TRPA Permit Data

***Get Data***
> TRPA permit data is exported from accela nightly then stored in colleciton.sde enterprise geodatabase and published to the trpa server as the web service below

In [ ]:
# web service url
permitTable = "https://maps.trpa.org/server/rest/services/Permit_Records/MapServer/1"
# get permit data as a dataframe
dfTRPAPermit = get_fs_data(permitTable)

In [ ]:
## TRPA Permit Data Engineering
dfTRPAPermit.info()

***Transformation***

In [ ]:
df = dfTRPAPermit

# final fields for all permit dataframes
fields = ['APN', 'Address', 'Jurisdiction', 'Permit_ID', 
          'Permit_Type','Permit_Category', 'Permit_Status',  'Description',
          'Applied_Date', 'Issued_Date', 'PreGrade_Date', 'Finaled_Date'
          ]

# # set fields
column_mapping = {
'Accela_ID' : 'Permit_ID',
'Detailed_Description' : 'Description',
'Record_Status' : 'Permit_Status',
'Accela_CAPType_Name' : 'Permit_Type',
'File_Date' : 'Applied_Date'
}

# rename columns based on dictionary
df = renamecolumns(df, column_mapping, False)

# add missing fields
for field in fields:
    # if field not in dataframe add it
    if field not in df.columns:
        # insert new column
        df[field] = None
# limit to the final fields
df = df[fields]
# add jurisdiction value
df.Jurisdiction = "TRPA"
df.info()


***Processing***

In [ ]:
# print out unique Record_Status values one at a time
for description in dfTRPAPermit.Detailed_Description.unique():
    print(description)

In [ ]:
# print out unique Record_Status values one at a time
for permittype in dfTRPAPermit.Accela_CAPType_Name.unique():
    print(permittype)

In [ ]:
# print out unique Record_Status values one at a time
for status in dfTRPAPermit.Record_Status.unique():
    print(status)

In [ ]:
value_lookup = "resources\Value_Lookups.csv"
trpa_reportingcategory_lookup = import_lookup_dictionary(value_lookup,'key','value','Jurisdiction','TRPA','FieldName','Reporting_Category')
trpa_permittype_lookup        = import_lookup_dictionary(value_lookup,'key','value','Jurisdiction','TRPA','FieldName','Permit_Type')
trpa_permitstatus_lookup      = import_lookup_dictionary(value_lookup,'key','value','Jurisdiction','TRPA','FieldName','Permit_Status')

In [ ]:
# Update fields from lookup dictionaries
df['Reporting_Category'] = df['Reporting_Category'].map(trpa_reportingcategory_lookup)
df['Permit_Type'] = df['Permit_Type'].map(trpa_permittype_lookup)
df['Permit_Status'] = df['Permit_Status'].map(trpa_permitstatus_lookup)

#### City of South Lake Tahoe Permit Data

***Get Data***

In [ ]:
## City of South Lake Tahoe Permit data was sent over by Ryan Malhoski on 4/9/2021
dfCSLTPermit = read_file("data\PermitData_CSLT_040924.csv")

In [ ]:
dfCSLTPermit.info()

***Transformation***

In [ ]:
# drop existing 'Address' field
df = dfCSLTPermit.drop('Address', axis=1)

# final fields for all permit dataframes
fields = ['APN', 'Address', 'Jurisdiction', 
          'Permit_ID', 'Permit_Type','Permit_Status', 'Description',
          'Applied_Date', 'Issued_Date', 'Finaled_Date'
          ]

# # set fields
column_mapping = {
            'Parcel ID': 'APN',
            'Location Address':'Address',
            'Permit Number' : 'Permit_ID',
            'Note Text' : 'Description',
            'Status' : 'Permit_Status',
            'Permit Type' : 'Permit_Type',
            'Permit Issue Date' : 'Applied_Date',
            'Certificate Issue Date': "Finaled_Date"
            }

# rename columns based on dictionary
df = renamecolumns(df, column_mapping,False)

# add missing fields
for field in fields:
    # if field not in dataframe add it
    if field not in df.columns:
        # insert new column
        df[field] = None
# limit to the final fields
df = df[fields]
# add jurisdiction value
df.Jurisdiction = "CSLT"
df.info()

In [ ]:
# APN is a PPNO format in the CSLT data, and also contains EL old naming convetion (-0)
# need to format to xxx-xxx-xxx and filter any odd values (e.g. 500 series)
# get rid of 100's and 500's series, and format to xxx-xxx-xxx, also remove any that start with strings
# strip off trailing spaces
df.APN = df.APN.str.replace(' ', '') 


***Processing***

In [ ]:
# potential values for Permit Type
# 
# get unique permit types
for permittype in dfCSLTPermit["Permit Type"].unique():
    print(permittype)

#### El Dorado County Permit Data
>  there are two files, one for all TRPA files and one for all files in our geographic area, including TRPA files and EDC files. 

***Get Data***

In [ ]:
## El Dorado Permit data representing all files in our geographic area
## exported by Ken Kasman on 4/1/2021 from their Trakit database
dfElDoPermit = read_file("data\PermitData_ElDorado_040124.csv")
dfElDoPermit.info()

***Transformation***

In [ ]:
# drop existing 'Address' field
df = dfElDoPermit

# final fields for all permit dataframes
fields = ['APN', 'Address', 'Jurisdiction', 
          'Permit_ID', 'Permit_Type','Permit_Status','Description',
          'Applied_Date', 'Issued_Date', 'Finaled_Date'
          ]

# # set fields
column_mapping = {
            'SITE_APN' : 'APN',
            'SITE_ADDR':'Address',
            'Permit Number' : 'Permit_ID',
            'DESCRIPTION' : 'Description',
            'STATUS' : 'Permit_Status',
            'PERMITTYPE' : 'Permit_Type',
            'APPLIED' : 'Applied_Date',
            'ISSUED'  : 'Issued_Date',
            'FINALED' : "Finaled_Date"
            }

# rename columns based on dictionary
df = renamecolumns(df, column_mapping, False)

# add missing fields
for field in fields:
    # if field not in dataframe add it
    if field not in df.columns:
        # insert new column
        df[field] = None
# limit to the final fields
df = df[fields]
# add jurisdiction value
df.Jurisdiction = "EL"
df.info()

In [ ]:
for permittype in dfElDoPermit["PERMITTYPE"].unique():
    print(permittype)

In [ ]:
# get lookup dictionary
lookupTable = read_file("resources/lookup_reporting_category.csv")
lookupTable["Reporting Category"].unique()


***Processing***

#### Placer County Permit Data

***Get Data***

In [ ]:
## Placer Permit Data Comes in monthly via email, and gets saved to the folder below.
## The code below will merge all the files in the folder into a single file, return a dataframe, and export to csv

# folder with the CSV files
folder_path = r"F:\Research and Analysis\Local Jurisdiction MOU data collection\Placer MOU Files\Placer"
# List to hold the DataFrames
dfs = []

# Loop through the files in the folder and identify CSV files
for file_name in os.listdir(folder_path):
    # Construct the full file path
    file_path = os.path.join(folder_path, file_name)
    # Read the CSV file into a DataFrame and append to the list
    df = pd.read_excel(file_path)
    # Append the DataFrame to the list
    dfs.append(df)
# Concatenate all DataFrames into a single DataFrame
final_df = pd.concat(dfs, ignore_index=True)
# Add today's date at the end of the file name _MMDDYY
today = pd.Timestamp.today().strftime("%m%d%y")
# Export the final DataFrame to a CSV file
final_df.to_csv("data\PermitData_Placer_" + today + ".csv", index=False)

In [ ]:
## Placer Permit data explained above. 
dfPlacerPermit =read_file("data\PermitData_Placer_040924.csv")

In [ ]:
dfPlacerPermit.info()

In [ ]:
dfPlacerPermit.head()

***Transformation***
> hyperlink to Placer Accela record can be bulit using SERV_PROD_CODE, B1_PER_ID1, B1_PER_ID2, B1_PER_ID3
* https://permits.placer.ca.gov/CitizenAccess/Cap/CapDetail.aspx?Module=TRPA&TabName=TRPA&capID1=16CAP&capID2=00000&capID3=0036O&agencyCode=PLACERCO

In [ ]:
# create lookup dictionary
lookupTable = read_file("resources/PL_lookup_reporting_category.csv")
lookupTable["Reporting Category"].unique()


***Processing***

#### Merge

In [ ]:
# merege the processed dfs
df = pd.concat([dfTRPA, dfCSLT, dfEL, dfPL], axis=0)

#### Load

In [ ]:
df.to_csv("data\PermitData.csv")

## Cumulative Accounting Data Engineering

#### Existing Development Rights

***Get Data***

In [ ]:
## get 2022 development units
devhistoryURL = "https://maps.trpa.org/server/rest/services/Existing_Development/MapServer/2"
parcelUnits12 = get_fs_data_spatial_query(devhistoryURL, "Year = 2012")
# parcelUnits18 = get_fs_data_spatial_query(devhistoryURL, "Year = 2018")
# parcelUnits19 = get_fs_data_spatial_query(devhistoryURL, "Year = 2019")
# parcelUnits20 = get_fs_data_spatial_query(devhistoryURL, "Year = 2020")
# parcelUnits21 = get_fs_data_spatial_query(devhistoryURL, "Year = 2021")
parcelUnits22 = get_fs_data_spatial_query(devhistoryURL, "Year = 2022")
parcelUnits23 = get_fs_data_spatial_query(devhistoryURL, "Year = 2023")

In [ ]:
# parcel history
url = 'https://maps.trpa.org/server/rest/services/AllParcels/MapServer/3'
dfParcelHistory = get_fs_data(url)

In [ ]:
# select all rows where b2023 active = 1 and all other fields = 0
dfParcel2023 = dfParcelHistory[(dfParcelHistory['b2023Active'] == 1) & 
                               (dfParcelHistory['b2022Active'] == 0) & 
                               (dfParcelHistory['b2021Active'] == 0) & 
                               (dfParcelHistory['b2020Active'] == 0) & 
                               (dfParcelHistory['b2019Active'] == 0) & 
                               (dfParcelHistory['b2018Active'] == 0) & 
                               (dfParcelHistory['b2012Active'] == 0)]
dfParcel2023.info()
list2023 = dfParcel2023['APN'].tolist()

In [ ]:
# export to data\parcelUnits_currentdate.csv with date stamp
today = pd.Timestamp.today().strftime("%m%d%y")
parcelUnits12.to_csv("data\parcelUnits12_" + today + ".csv", index=False)
# parcelUnits18.to_csv("data\parcelUnits18_" + today + ".csv", index=False)
# parcelUnits19.to_csv("data\parcelUnits19_" + today + ".csv", index=False)
# parcelUnits20.to_csv("data\parcelUnits20_" + today + ".csv", index=False)
# parcelUnits21.to_csv("data\parcelUnits21_" + today + ".csv", index=False)
parcelUnits22.to_csv("data\parcelUnits22_" + today + ".csv", index=False)
parcelUnits23.to_csv("data\parcelUnits23_" + today + ".csv", index=False)

In [ ]:
# Commercial data changes
dfCFA_Change            = pd.read_excel("data/CFA Changes.xlsx", sheet_name=0, skiprows=1)
dfCFA_ChangesLTinfo     = pd.read_excel("data/Working CFA Changes from LT Info Verifications.xlsx")
# drop any column with "Unamed" in the name
dfCFA_ChangesLTinfo     = dfCFA_ChangesLTinfo.loc[:, ~dfCFA_ChangesLTinfo.columns.str.contains('^Unnamed')]

# Residential data changes
parcelUnits12_Updated   = read_file("data/Updated_2012Analysis.csv")
dfRES_Change            = pd.read_excel("data/Residential Cleanup.xlsx")

# Tourist data changes
dfTAU_Change            = read_excel("data\Commercial and TAU Cumulative Accounting.xlsx", 0)

In [ ]:
# replace CFA with the updated values
parcelUnits12.CFA_SQFT = parcelUnits12_Updated.CFA_SQFT

In [ ]:
parcelUnits12.CommercialFloorArea_SqFt.sum()

***Transformation***

> Commercial/TAU changes – this file includes the changes tracked for commercial and TAU changes from ~2013 to current. There are projects listed that are completed, and others that have been permitted but not completed. Need to reconcile city-permitted permits from CSLT.

***Proecssing***

In [ ]:
dfParcelNew

In [ ]:

# compare where b20022 is 1 and b2023 is 0
dfParcelOld = dfParcelHistory[(dfParcelHistory['b2022Active'] == 1) & 
                               (dfParcelHistory['b2023Active'] == 0)]

# compare where b20022 is 0 and b2023 is 1
dfParcelNew = dfParcelHistory[(dfParcelHistory['b2022Active'] == 0) & 
                               (dfParcelHistory['b2023Active'] == 1)]

In [ ]:

# merge parcel22 and parcel23 on APN
dfParcelMerge = pd.merge(parcelUnits22, parcelUnits23, on='APN', how='outer', indicator=True)
# left only and right only
dfParcelMissing = dfParcelMerge[dfParcelMerge['_merge'] != 'both']

dfParcelMissing.info()

# left only
dfParcelMissingLeft = dfParcelMerge[dfParcelMerge['_merge'] == 'left_only']
dfParcelMissingLeft.info()

# sum of Residential_Units_x
dfParcelMissingLeft.Residential_Units_x.sum() 

# right only
dfParcelMissingRight = dfParcelMerge[dfParcelMerge['_merge'] == 'right_only']   
# dfParcelMissingRight.Residential_Units_y.sum() 

In [ ]:
# spatial join of dfParcelMissingLeft and dfParcelMissingRigh
sdf = parcelUnits22.spatial.join(parcelUnits23, how='inner', op='intersects')



In [ ]:
# arcpy spatial join
import arcpy

# arcpy spatial join of parcelUnits22 and parcelUnits23
df = arcpy.spatial


In [ ]:
# filter to rows where APN_left and APN_right !=
sdf2 = sdf[sdf.APN_left != sdf.APN_right]
sdf2.info()


In [ ]:
# spatial overlay of parcelUnits22 and parcelUnits23
sdf = parcelUnits22.spatial.overlay(parcelUnits23, how='union')

#### Deed Restrictions
> Deed restricted unit research needs to be merged with LTinfo housing deed restricitons and parcel unit data from 2022

***Get Data***

In [81]:
dfDeedUnits  = read_excel("data\Housing_Deed_Restrcitions.xlsx", 0)
dfDeedLTinfo = pd.read_json("https://laketahoeinfo.org/WebServices/GetDeedRestrictedParcels/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")

In [82]:
dfDeedUnits

,File_Number,APN,Deed_Restriction_Type,Units,Date_Type,LTinfo_Transaction_ID,LTInfo_Deed_Restriction,DR_Desc,Center,Notes,Issued Date,Acknowledged Date,Pre-Grade Date,Finaled Date
0,ERSP2022-1998,023-151-030,Achievable,1,"Permitted, Not Constructed",TRPA-ALLOC-2329,NaN,NaN,Non-Center,NaN,2023-10-09,NaT,NaT,NaT
1,20021919STD,023-162-017,Affordable,18,Constructed,NaN,DOC-2005-0023565-00,The eighteen (18) residential units on this pr...,OLD,Sky Forest Apartments,2003-04-23,2005-04-14,2005-04-14,2005-04-14
2,ERSP2020-2084,023-231-003,Achievable,70,"Reserved, Application in review",TRPA-ALLOC-1333,NaN,NaN,Center,"Crossings at the ""Y"" South Lake Tahoe",NaT,NaT,NaT,NaT
3,19980083STD,023-393-031,Affordable,45,Constructed,NaN,98-0009502-00,45 units on property must be occupied by perso...,OLD,Tahoe Senior Plaza,1998-04-27,1998-06-08,1998-06-11,1999-10-15
4,CORR2012-0143,023-632-005,Affordable,1,Constructed,NaN,DOC-2011-0058156-00,City of South Lake Tahoe deed restriction rest...,OLD,800 Taylor Way,2012-02-15,2012-03-22,2012-03-22,2012-03-22
5,ERSP2022-2018,023-655-012,Achievable,1,"Permitted, Not Constructed",TRPA-ALLOC-2119,2023-0008087,The accessory dwelling unit (ADU) on APN 023-6...,Non-Center,NaN,2023-04-19,NaT,NaT,NaT
6,ERSP2022-1992,025-041-023,Affordable,19,"Permitted, Not Constructed",TRPA-ALLOC-1871,2023-0013218,"To partially satisfy TRPA's March 22, 2023, co...",Local Jurisdiction Pool,LTCC Dorms,2023-03-22,2023-05-17,2023-05-18,NaT
7,NaN,025-041-023,Affordable,21,"Reserved, Application not submitted",TRPA-ALLOC-1872,NaN,NaN,Non-Center,"LTCC Dorms, Phase 2",NaT,NaT,NaT,NaT
8,ERSP2022-1992,025-041-023,Affordable,1,"Permitted, Not Constructed",TRPA-ALLOC-2274,2023-0013218,"To partially satisfy TRPA's March 22, 2023, co...",Local Jurisdiction Pool,NaN,2023-03-22,2023-05-17,2023-05-18,NaT
9,19911592STD,025-241-012,Affordable,28,Constructed,NaN,Doc# 014391,Deed restriction documenting a 28 unit afforda...,OLD,Tahoe Pines Apartments,1994-05-04,1994-05-12,1994-05-12,2012-08-16


In [ ]:
dfDeedUnits.Units.sum()

In [ ]:
dfDeedLTinfo.info()

In [ ]:
# get unique values for deed restrcition type
dfDeedLTinfo.DeedRestrictionType.unique()

# filter to Affordable, Achievable, and Moderate
dfDeedLTinfo = dfDeedLTinfo[dfDeedLTinfo.DeedRestrictionType.isin(['Affordable Housing', 'Moderate Income Housing', 'Achievable Housing'])]  

# count of total records
dfDeedLTinfo.shape[0]

In [ ]:
parcelUnits22.info()

In [ ]:
dfDeedUnitsMerge = dfDeedUnits.merge(dfDeedLTinfo, on='APN', how='outer', indicator=True)

In [ ]:
dfDeedUnitsMerge._merge.value_counts()

In [ ]:
dfDeedLTinfo[dfDeedLTinfo.duplicated(subset=['APN','DeedRestrictionType'], keep=False)].sort_values('APN').to_csv("HousingDeedRestrictions_LTinfo_Duplicates.csv")

In [ ]:
# identify duplicates unique by APN and 
dfDeedUnits[dfDeedUnits.duplicated(subset=['APN', 'Deed_Restriction_Type','Units'], keep=False)]

In [ ]:
# identify duplicates
dfDeedUnitsMerge[dfDeedUnitsMerge.duplicated(subset=['APN'], keep=False)].sort_values(by='APN')

In [ ]:
dfDeedUnitsMerge.to_csv("HousingDeedRestrictions_All.csv")

In [ ]:
# merge the deed restricted units with the parcel units
dfDeedUnits_ParcelUnits  = dfDeedUnits.merge(parcelUnits22, on='APN', how='left')
# merge the deed restricted units with the parcel units
dfDeedLTinfo_ParcelUnits = dfDeedLTinfo.merge(parcelUnits22, left_on='APN', right_on='APN', how='left')


In [ ]:
dfDeedLTinfo_ParcelUnits.info()

In [ ]:
dfDeedLTinfo_ParcelUnits.Residential_Units.sum()

#### ADU Tracking
> ADU permit tracking from TRPA and othe Jurisdictions. There is a need to establish a system of record for this information (LT Info). This is similar to the Residential Bonus Unit data and there’s crossover on some of these, where a bonus unit was used to create an ADU, but you can have an ADU without requiring a bonus unit, and you can use a bonus unit without it being an ADU… 

***Get Data***

In [ ]:
dfADU = read_excel("data\ADU Tracking.xlsx", 0)

In [ ]:
dfADU

#### Allocations
> This file includes all of the allocations that have been tracked in LT Info, and adds in whether the subject parcel has been issued a BMP/SCC certificate and/or whether Air Quality/Mobility Mitigation fees (for added VMT) or Water Quality Mitigation fees (for added coverage) have been paid. 

In [ ]:
allocations = read_excel("data\Allocation_Tracking.xlsx", 0)

#### Transactions with Inactive APNs

In [ ]:
inactiveParcels = read_file("data\Transactions_InactiveParcels.csv")

## QA Process

> Process to 